目前还差对数据进行清洗  
连续两次爬取的数据出现部分重叠，若大部分重叠，则爬取次数减少，意思到位就行  
最终输出连接前端？

图形位置:add()里的center参数  
标题位置:set_global_opts(title_opts=opts.TitleOpts(title="比例",pos_top=120))  
标注位置:set_global_opts(legend_opts=opts.LegendOpts(pos_left="20%"))  
pie半径：radius中的百分比是对应整个图形页面

In [69]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from pyecharts.charts import Pie,Bar,Map,Grid,Line,Page,Timeline
from pyecharts import options as  opts
from pyecharts.globals import ThemeType
import mysql.connector
from IPython.display import display
import jieba.analyse
import re
from pyecharts.charts import WordCloud
from pyecharts.globals import SymbolType
from snownlp import SnowNLP

In [34]:
def welcomed_reviews():#最受欢迎评论
    movies = []
    stars = []
    for i in range(0,100,20):
        headers={
            'User-Agent':UserAgent().random
        }
        url = "https://movie.douban.com/review/best/?start=%s"%str(i)
        resp = requests.get(url,headers=headers)
        soup = BeautifulSoup(resp.text,'lxml')
        tar_movies = soup.find_all('a',attrs={"class":"subject-img"})
        tar_stars = soup.find_all('header',attrs={'class':'main-hd'})
        for tar in tar_movies:
            movies.append(tar.img.get('title'))
        for tar in tar_stars:
            class_ = tar.find('span').get('class')
            stars.append(class_[0][7])
    return movies,stars

In [35]:
def get_hd_wlcmed_rvws():
    movies,stars = welcomed_reviews()

    M_S = {}
    M_T = {}
    for i in range(len(movies)):
        if(stars[i] == 't'):
            continue
        movie = movies[i]
        if not movie in M_S:
            M_T[movie] = 1
            M_S[movie] = round(float(stars[i]),1)
        else:
            M_S[movie] = round(((M_S[movie] * M_T[movie] + int(stars[i]))/(M_T[movie] + 1)),1)
            M_T[movie] += 1
    M_S_T = []
    for movie in list(M_S.keys()):
        M_S_T.append([movie,M_S[movie],M_T[movie]])
    M_S_T = sorted(M_S_T,key = lambda x:x[2],reverse=True)

    movies=[]
    stars=[]
    times=[]
    for i in range(len(M_S_T)):
        movies.append(M_S_T[i][0])
        stars.append(M_S_T[i][1])
        times.append(M_S_T[i][2])
    
    return movies,stars,times

In [82]:
def show_wlcmed_rvws_percent(movies,times):
    result_list = [(i,j) for i,j in zip(movies,times)]
    pie = Pie(init_opts=opts.InitOpts(theme = ThemeType.DARK))
    pie.add(series_name='电影',
        center=['50%','35%'],
        data_pair = result_list,
        rosetype='radius',
        radius='40%',
        )
    pie.set_global_opts(title_opts=opts.TitleOpts(title="电影比例",
                    pos_top=120),
                        )
    pie.set_series_opts(tooltip_opts=opts.TooltipOpts(trigger='item',formatter='{a} <br/>{b}:{c} ({d}%)'))
    return pie
def show_wlcmed_rvws_percent_star(movies,stars):
    bar = Bar(init_opts=opts.InitOpts(theme = ThemeType.DARK))
    bar.add_xaxis(movies)
    bar.add_yaxis('星级', stars)
    bar.set_global_opts(
            title_opts=opts.TitleOpts(title="平均评分"),
            xaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(rotate=-40)),
            legend_opts=opts.LegendOpts(orient='vertical',pos_top = '60%'),
            datazoom_opts=opts.DataZoomOpts(pos_top = '92%'),
        )
    bar.set_series_opts(tooltip_opts=opts.TooltipOpts(trigger='item',formatter='{a} <br/>{b}:{c}星'))
    return bar

In [37]:
m,s,t = get_hd_wlcmed_rvws()

In [ ]:
pie = show_wlcmed_rvws_percent(m,t)
bar = show_wlcmed_rvws_percent_star(m,s)
grid = Grid(init_opts = opts.InitOpts(width = "1000px",height = "1000px"))
grid.add(bar,grid_opts=opts.GridOpts(pos_top='65%',pos_bottom = '20%'))
grid.add(pie,grid_opts=opts.GridOpts())
grid.render_notebook()

爬取最近热评100条，选出平均评论数最多，星级最高的影视剧：长相思 第二季

配置cookie，初始化数据库密码

In [2]:
my_cookie = "gr_user_id=365152de-2f6f-445a-ad82-8b03064b595a; douban-fav-remind=1; ll=\"118318\"; bid=2uXnUuhqjFE; __yadk_uid=nJZfvl1Vf2Ov1qrYovKCWwctK3HSdqnW; _vwo_uuid_v2=DDC4449A754A4B9088C27B3678A9D1F41|9ba91b0e05a97c0b6baa8e2cc655397a; _pk_id.100001.4cf6=0ab5feb6f793fe25.1720514583.; __utmz=30149280.1720528388.5.2.utmcsr=baidu|utmccn=(organic)|utmcmd=organic; ct=y; push_doumail_num=0; push_noty_num=0; __utmv=30149280.28182; __utmz=223695111.1720849003.8.3.utmcsr=douban.com|utmccn=(referral)|utmcmd=referral|utmcct=/; dbcl2=\"281829339:36PhNryeN7A\"; ck=3szs; ap_v=0,6.0; __utmc=30149280; __utmc=223695111; frodotk_db=\"444bc41de2f2b227cce34c59a8d81675\"; __utma=30149280.517235761.1667117881.1720918321.1720922070.13; __utmb=30149280.0.10.1720922070; __utma=223695111.1166054037.1667117881.1720918321.1720922070.10; __utmb=223695111.0.10.1720922070; _pk_ref.100001.4cf6=%5B%22%22%2C%22%22%2C1720922070%2C%22https%3A%2F%2Fwww.douban.com%2F%3Fp%3D5%22%5D; _pk_ses.100001.4cf6=1"
standard_provinces = {
    '河北':'河北省',
    '山西':'山西省',
    '辽宁':'辽宁省',
    '吉林':'吉林省',
    '黑龙江':'黑龙江省',
    '江苏':'江苏省',
    '浙江':'浙江省',
    '安徽':'安徽省',
    '福建':'福建省',
    '江西':'江西省',
    '山东':'山东省',
    '河南':'河南省',
    '湖北':'湖北省',
    '湖南':'湖南省',
    '广东':'广东省',
    '海南':'海南省',
    '四川':'四川省',
    '贵州':'贵州省',
    '云南':'云南省',
    '陕西':'陕西省',
    '甘肃':'甘肃省',
    '青海':'青海省',
    '台湾':'台湾省',
    '内蒙古':'内蒙古自治区',
    '广西':'广西壮族自治区',
    '西藏':'西藏自治区',
    '宁夏':'宁夏回族自治区',
    '新疆':'新疆维吾尔自治区',
    '北京':'北京市',
    '天津':'天津市',
    '上海':'上海市',
    '重庆':'重庆市',
    '香港':'香港特别行政区',
    '澳门':'澳门特别行政区'
}
db_config = {
    'host' : 'localhost',
    'user' : 'root',
    'password' : '',
    'database' : 'movie_reviews'
}

In [3]:
def init_password():
    db_config['password'] = input().strip()

In [4]:
def short_reviews(type,sort):#P for watched,N for watching, F for wishing
    stars = []
    reviews = []
    provinces = []
    times = []
    print('----------type:%s----------'%type)
    print('----------sort:%s----------'%sort)
    if type == 'P':
        for i in range(0,600,20):
            if sort == 'time':
                url = "https://movie.douban.com/subject/36553434/comments?start=%s&limit=20&status=%s&sort=time"%(str(i),type)
            else:
                url = "https://movie.douban.com/subject/36553434/comments?start=%s&limit=20&status=%s&sort=new_score"%(str(i),type)
            headers = {
                'User-Agent':UserAgent().random,
                'Cookie': my_cookie
            }
            resp = requests.get(url,headers=headers)
            soup = BeautifulSoup(resp.text,'lxml')
            if soup.title.text == "没有访问权限":
                break
            divs = soup.find_all('div',attrs={'class':'comment'})
            if divs == None:
                break
            for div in divs:
                span1 = div.find('span',attrs={'class':'comment-info'})
                span2 = span1.find_all('span')
                star = span2[1].get('class')[0][7]
                if  star == '-':
                    continue #处理未打分用户
                province = span2[3].text.strip()
                if not province in standard_provinces:
                    continue
                stars.append(int(star))
                span3 = div.find('span',attrs={'class':'short'})
                reviews.append(span3.text.strip())
                provinces.append(standard_provinces[province])
                times.append(span1.find('span',attrs={'class':'comment-time'}).text.strip())
        return stars,provinces,times,reviews
    elif type == 'N':
        for i in range(0,600,20):
            if sort == 'time':
                url = "https://movie.douban.com/subject/36553434/comments?start=%s&limit=20&status=%s&sort=time"%(str(i),type)
            else:
                url = "https://movie.douban.com/subject/36553434/comments?start=%s&limit=20&status=%s&sort=new_score"%(str(i),type)
            headers = {
                'User-Agent':UserAgent().random,
                'Cookie': my_cookie
            }
            resp = requests.get(url,headers=headers)
            soup = BeautifulSoup(resp.text,'lxml')
            if soup.title.text == "没有访问权限":
                break
            divs = soup.find_all('div',attrs={'class':'comment'})
            if divs == None:
                break
            for div in divs:
                span1 = div.find('span',attrs={'class':'comment-info'})
                span2 = span1.find_all('span')
                star = span2[0].get('class')[0][7]
                if  star == '-':
                    continue #处理未打分用户
                province = span2[2].text.strip()
                if not province in standard_provinces:
                    continue
                stars.append(int(star))
                span3 = div.find('span',attrs={'class':'short'})
                reviews.append(span3.text.strip())
                provinces.append(standard_provinces[province])
                times.append(span1.find('span',attrs={'class':'comment-time'}).text.strip())
        return stars,provinces,times,reviews
    else:
        for i in range(0,600,20):
            if sort == 'time':
                url = "https://movie.douban.com/subject/36553434/comments?start=%s&limit=20&status=%s&sort=time"%(str(i),type)
            else:
                url = "https://movie.douban.com/subject/36553434/comments?start=%s&limit=20&status=%s&sort=new_score"%(str(i),type)
            headers = {
                'User-Agent':UserAgent().random,
                'Cookie': my_cookie
            }
            resp = requests.get(url,headers=headers)
            soup = BeautifulSoup(resp.text,'lxml')
            if soup.title.text == "没有访问权限":
                break
            divs = soup.find_all('div',attrs={'class':'comment'})
            if divs == None:
                break
            for div in divs:
                span1 = div.find('span',attrs={'class':'comment-location'})
                province = span1.text.strip()
                if not province in standard_provinces:
                    continue
                provinces.append(standard_provinces[province])
                span2 = div.find('span',attrs={'class':'short'})
                reviews.append(span2.text.strip())
                times.append(div.find('span',attrs={'class':'comment-time'}).text.strip())
                stars.append(0)
        return stars,provinces,times,reviews

In [5]:
def upload(type,stars,provinces,times,reviews):
    try:
        conn = mysql.connector.connect(**db_config)

        if conn.is_connected():
            print('---------successfully connected---------')
            for i in range(len(stars)):
                cursor = conn.cursor()
                insert_query = "INSERT INTO reviews VALUES (%s, %s, %s, %s, %s);"

                data_to_insert = (type, stars[i], provinces[i], times[i], reviews[i])

                cursor.execute(insert_query, data_to_insert)

            conn.commit()


    except mysql.connector.Error as err:
        print(f"---------Error: {err}---------")

    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('---------MySQL connection closed---------')

In [6]:
def download(type):
    stars=[]
    provinces=[]
    times=[]
    reviews=[]    
    try:
        conn = mysql.connector.connect(**db_config)

        if conn.is_connected():
            print('---------successfully connected---------')
            cursor = conn.cursor()

            query = 'SELECT * FROM reviews WHERE reviews.type = %s'
            cursor.execute(query, [type])

            rows = cursor.fetchall()
            for row in rows:
                stars.append(row[1])
                provinces.append(row[2])
                times.append(row[3])
                reviews.append(row[4])


    except mysql.connector.Error as err:
        print(f"---------Error: {err}---------")

    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('---------MySQL connection closed---------')
    return stars,provinces,times,reviews

In [7]:
def delete():
    try:
        conn = mysql.connector.connect(**db_config)

        if conn.is_connected():
            print('---------successfully connected---------')
            cursor = conn.cursor()

            cursor.execute('DELETE FROM reviews;')

            conn.commit()


    except mysql.connector.Error as err:
        print(f"---------Error: {err}---------")

    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('---------MySQL connection closed---------')

In [8]:
def spider(type,sort):
    s,p,t,r = short_reviews(type,sort)
    upload(type,s,p,t,r)
def spider_multi_types(types,sorts):
    for type in types:
        for sort in sorts:
            spider(type,sort)

In [9]:
def clean(stars,provinces,times,reviews):
    length = len(stars)
    locs = []
    for i in range(length):
        for j in range(i+1,length):
            if stars[i] == stars[j] and provinces[i] == provinces[j] and times[i] == times[j] and reviews[i] == reviews[j]:
                if not i in locs:
                    locs.append(i)
    locs.reverse()
    if locs == None:
        return
    for loc in locs:
        del stars[loc]
        del provinces[loc]
        del times[loc]
        del reviews[loc]

In [10]:
def clean_db():
    P_s,P_p,P_t,P_r = download('P')
    N_s,N_p,N_t,N_r = download('N')
    F_s,F_p,F_t,F_r = download('F')
    clean(P_s,P_p,P_t,P_r)
    clean(N_s,N_p,N_t,N_r)
    clean(F_s,F_p,F_t,F_r)
    delete()
    upload('P',P_s,P_p,P_t,P_r)
    upload('N',N_s,N_p,N_t,N_r)
    upload('F',F_s,F_p,F_t,F_r)

In [11]:
def var(nums):
    val = 0
    length = len(nums)
    mean = sum(nums)/length
    for n in nums:
        val += (n-mean)**2
    return round(val/length,2)
def avg(nums):
    return round(sum(nums)/len(nums),2)

In [12]:
def hd(watched_provinces,watched_stars,
       watching_provinces,watching_stars,
       wishing_provinces
       ):
    P_len = len(watched_stars)
    N_len = len(watching_stars)
    F_len = len(wishing_provinces)
    result_pair = []
    result_pair.append(('看过',P_len))
    result_pair.append(('在看',N_len))
    result_pair.append(('想看',F_len))

    provinces_times = {}
    for p in watched_provinces:
        if not p in provinces_times:
            provinces_times[p] = 1
        else:
            provinces_times[p] += 1
    for p in watching_provinces:
        if not p in provinces_times:
            provinces_times[p] = 1
        else:
            provinces_times[p] += 1
            
    for p in wishing_provinces:
        if not p in provinces_times:
            provinces_times[p] = 1
        else:
            provinces_times[p] += 1
        
    avgs = [avg(watched_stars),avg(watching_stars)]
    varss = [var(watched_stars),var(watching_stars)]
    return result_pair,provinces_times,avgs,varss

In [13]:
def show_map(provinces_times,max_num):
    map = Map(init_opts=opts.InitOpts(theme = ThemeType.DARK))
    map.add("评论人数", [list(z) for z in zip(list(provinces_times.keys()),list(provinces_times.values()))], "china")
    map.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
    map.set_global_opts(title_opts=opts.TitleOpts(title="分布图"),
                        visualmap_opts=opts.VisualMapOpts(max_= max_num,pos_left = '10%',pos_bottom='10%'),
                        )
    return map
def show_rating(result_pair,centerx,centery,r,R):
    pie = Pie(init_opts=opts.InitOpts(theme = ThemeType.DARK))
    pie.add(series_name='电影',
            center=[centerx,centery],
            data_pair = result_pair,
            radius=[r,R]
        )
    pie.set_global_opts(title_opts=opts.TitleOpts(title="比例"),
                        legend_opts=opts.LegendOpts(),)
    pie.set_series_opts(tooltip_opts=opts.TooltipOpts(trigger='item',formatter='{a} <br/>{b}:{c} ({d}%)'),)
    return pie
def show_star(avgs,varss):
    bar = Bar(init_opts=opts.InitOpts(theme = ThemeType.DARK))
    bar.add_xaxis(['看过','在看'])
    bar.add_yaxis('星级', avgs)
    bar.add_yaxis('评分方差',varss)
    bar.set_global_opts(
            title_opts=opts.TitleOpts(title='打分情况'),
            legend_opts=opts.LegendOpts(),
            yaxis_opts=opts.AxisOpts(name="星级"),
            xaxis_opts=opts.AxisOpts(name="类型")
        )
    return bar

In [15]:
def get_keyword(text):
    keywords = jieba.analyse.textrank(
        sentence=text,
        topK=15,
        # allowPOS=['n','nz','ns', 'nr'],
        withWeight=False,  # 权重
        withFlag=False,    # 词性
    )
    return keywords

In [201]:
def hd_comment(comment):
    pattern = re.compile(r"[\u4e00-\u9fa5]+")
    words = re.findall(pattern, comment)
    text = ''
    for word in words:
        text += word
    stop_words = ['没有','感觉','看到','不能','希望','男人','时候', '防风', '知道','觉得','有点','清水','还有']
    words = jieba.lcut(text)
    text = ''
    for word in words:
        if not word in stop_words:
            text += word
    return text,words

In [207]:
def show_keywords_cloud(words):
    cloud = WordCloud(init_opts=opts.InitOpts(theme = ThemeType.DARK))
    cloud.add("", words, word_size_range=[20, 100], shape=SymbolType.DIAMOND)
    cloud.set_global_opts(title_opts=opts.TitleOpts(title="词云"))
    return cloud

In [18]:
def emotion(comments):
    scores = []
    for comment in comments:
        s = SnowNLP(comment)
        scores.append(s.sentiments)
    return scores

In [19]:
def show_emotion(P_scores,N_scores,F_scores):
    types_scores =[P_scores,N_scores,F_scores]
    P_attitude = [0,0,0]
    N_attitude = [0,0,0]
    F_attitude = [0,0,0]
    types_attitude = [P_attitude,N_attitude,F_attitude]
    for i in range(3):
        for score in types_scores[i]:
            if score > 0.85:
                types_attitude[i][0] += 1
            elif score > 0.55:
                types_attitude[i][1] += 1
            else:
                types_attitude[i][2] += 1
    sum_attitude = [P_attitude[i] + N_attitude[i] + F_attitude[i] for i in range(3)]
    list1 = [{'value': P_attitude[i],'percent':round(P_attitude[i]/sum_attitude[i],2)} for i in range(3)]
    list2 = [{'value': N_attitude[i],'percent':round(N_attitude[i]/sum_attitude[i],2)} for i in range(3)]
    list3 = [{'value': F_attitude[i],'percent':round(F_attitude[i]/sum_attitude[i],2)} for i in range(3)]
    c = (
        Bar(init_opts=opts.InitOpts(theme=ThemeType.DARK))
        .add_xaxis(['积极','中肯','消极'])
        .add_yaxis("看过", list1, stack="stack1", category_gap="50%")
        .add_yaxis("在看", list2, stack="stack1", category_gap="50%")
        .add_yaxis("想看", list3, stack="stack1", category_gap="50%")
        .set_series_opts(
            label_opts=opts.LabelOpts(
                position="right",
            )
        )
        .set_global_opts(
            title_opts=opts.TitleOpts(title='情感分析'),
            legend_opts=opts.LegendOpts(),
            yaxis_opts=opts.AxisOpts(name="占比"),
            xaxis_opts=opts.AxisOpts(name="态度")
        )
    )
    return c

In [41]:
def time_tend():
    times = []
    P_stars = []
    N_stars = []
    stars = {'P':P_stars,'N':N_stars}
    try:
        conn = mysql.connector.connect(**db_config)

        if conn.is_connected():
            print('---------successfully connected---------')
            cursor = conn.cursor()

            query = 'SELECT type,star,time FROM reviews ORDER by time ASC'
            cursor.execute(query)

            rows = cursor.fetchall()
            standard_time = None
            P_stars_sum = 0
            N_stars_sum = 0
            stars_sum = {'P':P_stars_sum,'N':N_stars_sum}
            P_time = 0
            N_time = 0
            stars_time = {'P':P_time,'N':N_time}
            count = 0
            for row in rows:
                if row[0] == 'F':
                    continue
                if standard_time == None:
                    standard_time = row[2]
                    stars_sum[row[0]] = 0
                    stars_time[row[0]] = 0
                if row[2][:14] == standard_time[:14]:
                    star = int(row[1])
                    stars_sum[row[0]] += star
                    stars_time[row[0]] += 1
                elif count == 0:
                    count = 0
                    times.append(standard_time[:13])
                    standard_time = None
                    for type in ['P','N']:
                        if stars_time[type] != 0:
                            stars[type].append(round(stars_sum[type] / stars_time[type],2))
                        else:
                            stars[type].append(None)
                else:
                    count += 1


    except mysql.connector.Error as err:
        print(f"---------Error: {err}---------")

    finally:
        if 'conn' in locals() and conn.is_connected():
            cursor.close()
            conn.close()
            print('---------MySQL connection closed---------')
    return times,P_stars,N_stars

In [185]:
def show_time_tend(times,P_stars,N_stars,days,good,mid,bad):
    tl = Timeline(init_opts=opts.InitOpts(theme = ThemeType.DARK))
    x_label = ['%s点'%(int(d)) for d in days]
    init_day = ''
    ps = [0 for i in range(24)]
    ns = [0 for i in range(24)]
    for i in range(len(times)):
        day = times[i][5:7] + times[i][8:10]
        time = int(times[i][11:13])
        if init_day == '':
            init_day = day
        elif init_day == day:
            ps[time] = P_stars[i]
            ns[time] = N_stars[i]
        else:
            init_day = ''
            line = Line()
            line.add_xaxis(x_label)
            line.add_yaxis(
                series_name="看过",
                y_axis=ps,
                yaxis_index=1,
            )
            line.add_yaxis(
                series_name="在看",
                y_axis=ns,
                yaxis_index=1,
            )
            line.set_series_opts(label_opts=opts.LabelOpts(is_show=False))
            bar = Bar(init_opts=opts.InitOpts(theme = ThemeType.DARK))
            bar.add_xaxis(x_label)
            bar.add_yaxis('好评',good)
            bar.add_yaxis('中评',mid)
            bar.add_yaxis('差评',bad)
            bar.set_global_opts(
                    title_opts=opts.TitleOpts(title='时段打分情况',subtitle='%s月%s日'%(times[i][5:7],times[i][8:10])),
                    legend_opts=opts.LegendOpts(),
                )
            bar.extend_axis(
                yaxis=opts.AxisOpts(
                    axislabel_opts=opts.LabelOpts(formatter="{value} 星"), interval=1,max_=5
                )
            )
            bar.set_series_opts(itemstyle_opts=opts.ItemStyleOpts(opacity=0.4),
                                label_opts=opts.LabelOpts(is_show=False),
                                yaxis_opts=opts.AxisOpts(axislabel_opts=opts.LabelOpts(formatter="{value} 次"),interval=2,max_=10)
                                )
            bar.overlap(line)
            tl.add(bar, day)
            ps = [0 for i in range(24)]
            ns = [0 for i in range(24)]
    return tl


In [20]:
init_password()

In [ ]:
'''types = ['P','N','F']
sorts = ['hot','time']
spider_multi_types(types,sorts)'''

In [ ]:
'''clean_db()'''

In [ ]:
P_s,P_p,P_t,P_r = download('P')
N_s,N_p,N_t,N_r = download('N')
F_s,F_p,F_t,F_r = download('F')

In [ ]:
comments = ''
for i in range(len(P_r)):
    comments += P_r[i]
for i in range(len(N_r)):
    comments += N_r[i]
for i in range(len(F_r)):
    comments += F_r[i]
text,words = hd_comment(comments)

In [209]:
text,words = hd_comment(comments)
keywords = get_keyword(text)
print(keywords)

['剧情', '期待', '演技', '相思', '女主', '好看', '喜欢', '演员', '角色', '权谋', '感情', '表哥', '故事', '观众', '节奏']


In [210]:
keywords = get_keyword(text)
keyword_time = {}
for keyword in keywords:
    keyword_time[keyword] = 0
for word in words:
    if word in keywords:
        keyword_time[word] += 1
ktk = list(keyword_time.keys())
ktt = list(keyword_time.values())
words = [(i,j) for i,j in zip(ktk,ktt)]

In [26]:
result_pair,provinces_times,avgs,varss = hd(P_p,P_s,N_p,N_s,F_p)

In [27]:
P_scores = emotion(P_r)
N_scores = emotion(N_r)
F_scores = emotion(F_r)

In [ ]:
times,P_stars,N_stars = time_tend()

In [64]:
days = ['0%s'%str(i) for i in range(10)]
for i in range(10,24):
    days.append('%s'%str(i))
good = [0 for i in range(24)]
mid = [0 for i in range(24)]
bad = [0 for i in range(24)]
for i in range(len(times)):
    pos = int(times[i][11:13])
    if P_stars[i] >= 4:
        good[pos] += 1
    elif P_stars[i] >= 2:
        mid[pos] += 1
    else:
        bad[pos] += 1
    if N_stars[i] >= 4:
        good[pos] += 1
    elif P_stars[i] >= 2:
        mid[pos] += 1
    else:
        bad[pos] += 1

In [28]:
emotion_bar = show_emotion(P_scores,N_scores,F_scores)

In [211]:
cloud = show_keywords_cloud(words)

In [30]:
map_cn = show_map(provinces_times,200)

In [31]:
pie = show_rating(result_pair,centerx='50%',centery='50%',r='30%',R='80%')

In [57]:
bar = show_star(avgs,varss)

In [186]:
tl = show_time_tend(times,P_stars,N_stars,days,good,mid,bad)

In [213]:
tl.render_notebook()

In [ ]:
page = Page(layout=Page.DraggablePageLayout) 
page.add(emotion_bar,tl,cloud,pie,map_cn,bar) 
page.render_notebook()